In [1]:
# import libraries

import numpy as np
import pandas as pd
import csv
import nltk
nltk.download()
import re

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
# read file

dataset = pd.read_csv(r"C:\Users\Ιακώβη\Downloads\IMDB_Dataset.csv")

In [3]:
# create and print a dataframe

df = pd.DataFrame(dataset)
print(df)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [4]:
# create and print a shortened version of the dataset

shortened_dataset = df.head(10000)
print(shortened_dataset)

                                                 review sentiment
0     One of the other reviewers has mentioned that ...  positive
1     A wonderful little production. <br /><br />The...  positive
2     I thought this was a wonderful way to spend ti...  positive
3     Basically there's a family where a little boy ...  negative
4     Petter Mattei's "Love in the Time of Money" is...  positive
...                                                 ...       ...
9995  Fun, entertaining movie about WWII German spy ...  positive
9996  Give me a break. How can anyone say that this ...  negative
9997  This movie is a bad movie. But after watching ...  negative
9998  This is a movie that was probably made to ente...  negative
9999  Smashing film about film-making. Shows the int...  positive

[10000 rows x 2 columns]


In [5]:
# find the shape of the shortened dataset

shortened_dataset.shape

(10000, 2)

In [6]:
# check the distribution of sentiment

shortened_dataset['sentiment'].value_counts()

positive    5028
negative    4972
Name: sentiment, dtype: int64

In [7]:
# check for null values

shortened_dataset.isnull().sum()

review       0
sentiment    0
dtype: int64

In [8]:
# create a label for reviews section and for sentiment section 

x = shortened_dataset['review']   
y = shortened_dataset['sentiment']  
print(x,y)

train_X=[]
test_X=[]

0       One of the other reviewers has mentioned that ...
1       A wonderful little production. <br /><br />The...
2       I thought this was a wonderful way to spend ti...
3       Basically there's a family where a little boy ...
4       Petter Mattei's "Love in the Time of Money" is...
                              ...                        
9995    Fun, entertaining movie about WWII German spy ...
9996    Give me a break. How can anyone say that this ...
9997    This movie is a bad movie. But after watching ...
9998    This is a movie that was probably made to ente...
9999    Smashing film about film-making. Shows the int...
Name: review, Length: 10000, dtype: object 0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 10000, dtype: object


In [9]:
# split train from test set

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state = 42)

In [10]:
# find the shape of train and test set

x_train.shape, x_test.shape

((8000,), (2000,))

In [11]:
# convert to lists (iterable objects) for the following data preprocessing loop

train_x = list(x_train)
test_x = list(x_test)
train_y = list(y_train)
test_y = list(y_test)

In [12]:
# data preprocessing 

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')

for i in range(0, len(train_x)):
    review = re.sub('[^a-zA-Z]', ' ', train_x[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)

In [13]:
# check a random example of preprocessed text

train_X[3]

'despite critic ensue enjoyed immensely precisely eyecandy side gender spectrum soderberg done artsy hard edge stuff oscar top game ocean light commercial fluffy steve day midway generally fan zeta jones even must admit kate stunning movie ending scream upcoming trequel one million flock see minute george brad matt parlay clooney dig lago di como swindle rich bad guy tolerated installment lord ring ask drool clooney salt pepper lid one time'

In [14]:
# tf-idf vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer()

# apply tf-idf to training data
X_train_tf = tf_idf.fit_transform(train_X)

In [15]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 8000, n_features: 41860


In [16]:
# transform testing data into tf-idf matrix
X_test_tf = tf_idf.transform(test_x)

In [17]:
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

n_samples: 2000, n_features: 41860


In [18]:
# support vector machine classifier
from sklearn import svm
from sklearn.svm import LinearSVC

svm_classifier = svm.SVC(kernel='linear')

# apply SVM classifier to training data
svm_classifier.fit(X_train_tf, train_y)

SVC(kernel='linear')

In [19]:
# predict the test data
y_pred = svm_classifier.predict(X_test_tf)

In [20]:
# analyze the results
from sklearn import metrics

print(metrics.classification_report(test_y, y_pred, target_names=['Negative', 'Positive']))

              precision    recall  f1-score   support

    Negative       0.92      0.77      0.84       996
    Positive       0.80      0.94      0.87      1004

    accuracy                           0.85      2000
   macro avg       0.86      0.85      0.85      2000
weighted avg       0.86      0.85      0.85      2000

